In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On Simple English wiki, POS tagged

sf = open(datapath+'/Corpora/wiki/simple_20200601/Tagged/simple_20200601_tagged_clean.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

April|NNP is|VBZ the|DT fourth|JJ month|NN of|IN the|DT year|NN and|CC comes|VBZ between|IN March|NNP and|CC May|NNP 



It|PRP is|VBZ one|CD of|IN four|CD months|NNS to|TO have|VB 30|CD days|NNS 



April|NNP always|RB begins|VBZ on|IN the|DT same|JJ day|NN of|IN week|NN as|IN July|NNP and|CC additionally|RB January|NNP in|IN leap|NNP years|NNS 



In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

from nltk.corpus.reader.util import read_line_block

In [5]:
simp_t = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/Tagged','simple_20200601_tagged_clean.txt',
                            word_tokenizer = WhitespaceTokenizer(),
                            para_block_reader=read_line_block
                            )

In [6]:
# Import word and sentence generators

from generators import simp_word_gen

In [7]:
from nltk.util import ngrams, everygrams

In [8]:
i = 0
for sent in simp_t.sents():
    i += 1
    print(sent)
    if i > 5 :break

['April|NNP', 'is|VBZ', 'the|DT', 'fourth|JJ', 'month|NN', 'of|IN', 'the|DT', 'year|NN', 'and|CC', 'comes|VBZ', 'between|IN', 'March|NNP', 'and|CC', 'May|NNP']
['It|PRP', 'is|VBZ', 'one|CD', 'of|IN', 'four|CD', 'months|NNS', 'to|TO', 'have|VB', '30|CD', 'days|NNS']
['April|NNP', 'always|RB', 'begins|VBZ', 'on|IN', 'the|DT', 'same|JJ', 'day|NN', 'of|IN', 'week|NN', 'as|IN', 'July|NNP', 'and|CC', 'additionally|RB', 'January|NNP', 'in|IN', 'leap|NNP', 'years|NNS']
['April|NNP', 'always|RB', 'ends|VBZ', 'on|IN', 'the|DT', 'same|JJ', 'day|NN', 'of|IN', 'the|DT', 'week|NN', 'as|IN', 'December|NNP']
["April|NNP's", 'flowers|NNS', 'are|VBP', 'the|DT', 'Sweet|NNP', 'Pea|NN', 'and|CC', 'Daisy|NNP']
['Its|PRP$', 'birthstone|NN', 'is|VBZ', 'the|DT', 'diamond|NN']


In [10]:
i = 0
for sent in simp_word_gen(simp_t):
    i += 1
    print(sent)
    if i > 20 :break

April|NNP
is|VBZ
the|DT
fourth|JJ
month|NN
of|IN
the|DT
year|NN
and|CC
comes|VBZ
between|IN
March|NNP
and|CC
May|NNP
|^|
It|PRP
is|VBZ
one|CD
of|IN
four|CD
months|NNS


In [8]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [9]:
scorer = NgramAssocMeasures.poisson_stirling

In [14]:
tri_cf = TrigramCollocationFinder.from_words(simp_word_gen(simp_t))
tri_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [15]:
bi_cf = tri_cf.bigram_finder()
bi_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [16]:
print("\t", [" ".join(tup) for tup in bi_cf.nbest(scorer, 15)])

	 ['#Categoria|NN neglijarea|NN', '#Drugoplanowi|NNP bohaterowie|NN', '#EXO|NN KoKoBop|NN', '#El|NNP diagnòstic|NNP', '#Other|NN websites1|NN', '#Outros|NNP álbuns|NNS', '#סרביה|NNP ומונטנגרו|NNP', "'44|CD Timezone|NNP", "'96|CD LNCS|NN", "'|''-kew|NN-lye|NN '|''-nə|NN", '(|-LRB--CSU|NNP 7830ri|CD-Joachim|NNP', '(|-LRB--YE|NN ʿALĪ|NN', ')|-RRB--Peninsular|NNP War|NNP/Sixth|NNP', ')|-RRB--Proctor|NNP Vly|NNP', ')|-RRB--Second|JJ Coalition|NN/Egyptian|JJ']


In [17]:
bi_dict = {}
tri_dict = {}

In [18]:
for bigram in bi_cf.score_ngrams(scorer):    
    bi_dict[bigram[0]] = [bigram[0], bi_cf.ngram_fd[bigram[0]], bigram[-1]]
    
bi_dict

{('#Categoria|NN', 'neglijarea|NN'): [('#Categoria|NN', 'neglijarea|NN'),
  1,
  -25.976128283082176],
 ('#Drugoplanowi|NNP',
  'bohaterowie|NN'): [('#Drugoplanowi|NNP',
   'bohaterowie|NN'), 1, -25.976128283082176],
 ('#EXO|NN', 'KoKoBop|NN'): [('#EXO|NN', 'KoKoBop|NN'),
  1,
  -25.976128283082176],
 ('#El|NNP', 'diagnòstic|NNP'): [('#El|NNP', 'diagnòstic|NNP'),
  1,
  -25.976128283082176],
 ('#Other|NN', 'websites1|NN'): [('#Other|NN', 'websites1|NN'),
  1,
  -25.976128283082176],
 ('#Outros|NNP', 'álbuns|NNS'): [('#Outros|NNP', 'álbuns|NNS'),
  1,
  -25.976128283082176],
 ('#סרביה|NNP', 'ומונטנגרו|NNP'): [('#סרביה|NNP', 'ומונטנגרו|NNP'),
  1,
  -25.976128283082176],
 ("'44|CD", 'Timezone|NNP'): [("'44|CD", 'Timezone|NNP'),
  1,
  -25.976128283082176],
 ("'96|CD", 'LNCS|NN'): [("'96|CD", 'LNCS|NN'), 1, -25.976128283082176],
 ("'|''-kew|NN-lye|NN", "'|''-nə|NN"): [("'|''-kew|NN-lye|NN", "'|''-nə|NN"),
  1,
  -25.976128283082176],
 ('(|-LRB--CSU|NNP',
  '7830ri|CD-Joachim|NNP'): [('(|-

In [19]:
for trigram in tri_cf.score_ngrams(scorer):    
    tri_dict[trigram[0]] = [trigram[0], tri_cf.ngram_fd[trigram[0]], trigram[-1]]
    
tri_dict

{('(|-LRB--CSU|NNP',
  '7830ri|CD-Joachim|NNP',
  'Herrmann.jpgJoachim|NNP'): [('(|-LRB--CSU|NNP',
   '7830ri|CD-Joachim|NNP',
   'Herrmann.jpgJoachim|NNP'), 1, -25.976128283082176],
 ('(|-LRB--YE|NN',
  'ʿALĪ|NN',
  'iranicaonline.org|ADD'): [('(|-LRB--YE|NN',
   'ʿALĪ|NN',
   'iranicaonline.org|ADD'), 1, -25.976128283082176],
 (')|-RRB--mei2|NN',
  'Éméi|NNP',
  'Jianshih|NNP'): [(')|-RRB--mei2|NN',
   'Éméi|NNP',
   'Jianshih|NNP'), 1, -25.976128283082176],
 (')|-RRB-/Non|FW',
  'aver|FW',
  'nessuno|FW'): [(')|-RRB-/Non|FW',
   'aver|FW',
   'nessuno|FW'), 1, -25.976128283082176],
 ('+160|CD', '+360|CD', '+160|NN'): [('+160|CD', '+360|CD', '+160|NN'),
  1,
  -25.976128283082176],
 (',140,000|CD',
  '112,420,000|CD',
  '224,560,000|CD'): [(',140,000|CD',
   '112,420,000|CD',
   '224,560,000|CD'), 1, -25.976128283082176],
 (',205|NN',
  '205,515|CD',
  '247,987|CD'): [(',205|NN',
   '205,515|CD',
   '247,987|CD'), 1, -25.976128283082176],
 (',25|CD',
  'Elmasri|NNP',
  'Navathe26|NN'

In [20]:
bigram_df = pd.DataFrame.from_dict(bi_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

trigram_df = pd.DataFrame.from_dict(tri_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

bigram_df

,ngram,freq,poisson
"(#Categoria|NN, neglijarea|NN)","(#Categoria|NN, neglijarea|NN)",1,-2.597613e+01
"(#Drugoplanowi|NNP, bohaterowie|NN)","(#Drugoplanowi|NNP, bohaterowie|NN)",1,-2.597613e+01
"(#EXO|NN, KoKoBop|NN)","(#EXO|NN, KoKoBop|NN)",1,-2.597613e+01
"(#El|NNP, diagnòstic|NNP)","(#El|NNP, diagnòstic|NNP)",1,-2.597613e+01
"(#Other|NN, websites1|NN)","(#Other|NN, websites1|NN)",1,-2.597613e+01
...,...,...,...
"(and|CC, the|DT)","(and|CC, the|DT)",50301,-2.522274e+06
"(to|IN, the|DT)","(to|IN, the|DT)",62524,-3.011091e+06
"(is|VBZ, a|DT)","(is|VBZ, a|DT)",83635,-3.947372e+06
"(in|IN, the|DT)","(in|IN, the|DT)",157323,-7.612996e+06


In [21]:
ngram_df = bigram_df.append(trigram_df).sort_values('poisson', ascending=False).reset_index(drop=True)

In [22]:
ngram_df['len'] = ngram_df.ngram.apply(len)

In [23]:
ngram_df

,ngram,freq,poisson,len
0,"(#Categoria|NN, neglijarea|NN)",1,-2.597613e+01,2
1,"(Île|NN-Tudy|NN, 29086|CD)",1,-2.597613e+01,2
2,"(Í̱|NN, í̱|NN)",1,-2.597613e+01,2
3,"(Í̯|NN, í̯|NN)",1,-2.597613e+01,2
4,"(Í̧|NN, í̧|NN)",1,-2.597613e+01,2
...,...,...,...,...
23968257,"(and|CC, the|DT)",50301,-2.522274e+06,2
23968258,"(to|IN, the|DT)",62524,-3.011091e+06,2
23968259,"(is|VBZ, a|DT)",83635,-3.947372e+06,2
23968260,"(in|IN, the|DT)",157323,-7.612996e+06,2


In [25]:
# Pickle ngram_df - can re-use later with different approaches (ngram_df = pd.read_pickle() )
ngram_df.to_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_df_clean.pkl')

In [10]:
# ngram_df = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_df_clean.pkl')

In [26]:
len(simp_t.words())

30844031

In [55]:
# Stopwords from corpus - 50 most frequent
from nltk import FreqDist

fdist = FreqDist(simp_word_gen(simp_t, sent_mark=''))

stop = set( word for word, f in fdist.most_common(50))

In [42]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 '4|CD',
 ':|:',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'This|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'born|VBN',
 'by|IN',
 'can|MD',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'people|NNS',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'they|PRP',
 'the|DT',
 'to|IN',
 'to|TO',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'with|IN'}

In [11]:
import pickle

In [58]:
with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop_clean.pkl', 'wb') as pfile:
    pickle.dump(stop,pfile)

In [41]:
#with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop_clean.pkl', 'rb') as pfile:
#    stop = pickle.load(pfile)

In [12]:
from batcher import batcher  # Custom module with logic for assigning n-grams to batches, avoiding overlap

In [13]:
min_freq = 20

In [14]:
# Duplicate entries appearing for some reason. Removing here
ngram_df2 = ngram_df[ngram_df.freq >= min_freq].drop_duplicates().reset_index(drop=True)

ngram_df2

,ngram,freq,poisson,len
0,"(Grădina|NNP, Zoologică|NNP)",20,-6.059611e+02,2
1,"(Ving|NNP, Rhames|NNP)",20,-6.059611e+02,2
2,"(Karlovy|NNP, Vary|NNP)",20,-6.059611e+02,2
3,"(Gharb|NNP-Chrarda|NNP-Beni|NNP, Hssen|NNP)",20,-6.059611e+02,2
4,"(waystations|NNS, shuku|NN-eki|NN)",20,-6.073689e+02,2
...,...,...,...,...
216168,"(and|CC, the|DT)",50301,-2.522274e+06,2
216169,"(to|IN, the|DT)",62524,-3.011091e+06,2
216170,"(is|VBZ, a|DT)",83635,-3.947372e+06,2
216171,"(in|IN, the|DT)",157323,-7.612996e+06,2


Additional filtering of low-quality candidates

In [15]:
# Filtering patterns

_tagnums = re.compile("(\d+(\.\d+)?$)|(.+\|CD$)")  # All numeric, with optional decimal point OR anything tagged |CD
_nans    = re.compile("nan", re.I)                 # Occasional lists like (NaN, NaN, NaN)
_timerad = re.compile("[apf]\.?m.?", re.I)         # a.m. P.M, FM - time and radio station indicators

def tupallmatch(tupin, pattern):
    return all(pattern.match(w) for w in tupin)

def tupmatchlist(tupin, patterns):
    for w,p in zip(tupin, patterns):
        if p.match(w): pass
        else: return False
    return True

In [16]:
# initially;    216,173  
ngrams_filt = ngram_df2[~ngram_df2.ngram.apply(tupallmatch , args= [_tagnums])]
# all numeric;  208,479
ngrams_filt = ngrams_filt[~ngrams_filt.ngram.apply(tupmatchlist, args = [[_tagnums, re.compile('males\|NNS')]])]
# x|CD males;   208,475
ngrams_filt = ngrams_filt[~ngrams_filt.ngram.apply(tupallmatch , args= [_nans])]
# NaN NaN;      208,475
ngrams_filt = ngrams_filt[~ngrams_filt.ngram.apply(tupmatchlist, args = [[_tagnums, _timerad]])]
# Times, radio; 208,344

In [17]:
ngrams_filt = ngrams_filt.reset_index(drop=True)

ngrams_filt

,ngram,freq,poisson,len
0,"(Grădina|NNP, Zoologică|NNP)",20,-6.059611e+02,2
1,"(Ving|NNP, Rhames|NNP)",20,-6.059611e+02,2
2,"(Karlovy|NNP, Vary|NNP)",20,-6.059611e+02,2
3,"(Gharb|NNP-Chrarda|NNP-Beni|NNP, Hssen|NNP)",20,-6.059611e+02,2
4,"(waystations|NNS, shuku|NN-eki|NN)",20,-6.073689e+02,2
...,...,...,...,...
208339,"(and|CC, the|DT)",50301,-2.522274e+06,2
208340,"(to|IN, the|DT)",62524,-3.011091e+06,2
208341,"(is|VBZ, a|DT)",83635,-3.947372e+06,2
208342,"(in|IN, the|DT)",157323,-7.612996e+06,2


In [59]:
eval_count = 150000

ngram_eval = ngrams_filt[0:eval_count]

ngram_eval

,ngram,freq,poisson,len
0,"(Grădina|NNP, Zoologică|NNP)",20,-605.961128,2
1,"(Ving|NNP, Rhames|NNP)",20,-605.961128,2
2,"(Karlovy|NNP, Vary|NNP)",20,-605.961128,2
3,"(Gharb|NNP-Chrarda|NNP-Beni|NNP, Hssen|NNP)",20,-605.961128,2
4,"(waystations|NNS, shuku|NN-eki|NN)",20,-607.368914,2
...,...,...,...,...
149995,"(supporters|NNS, of|IN, the|DT)",47,-3262.258611,3
149996,"(episodes|NNS, and|CC)",64,-3262.274573,2
149997,"(as|IN, old|JJ, as|IN)",48,-3262.394255,3
149998,"(John|NNP, Kerry|NNP)",77,-3262.495991,2


In [60]:
batches, batch_count = batcher(ngram_eval.ngram, stopwords=stop, max_batches = 15)

batches

{('Grădina|NNP', 'Zoologică|NNP'): 1,
 ('Ving|NNP', 'Rhames|NNP'): 1,
 ('Karlovy|NNP', 'Vary|NNP'): 1,
 ('Gharb|NNP-Chrarda|NNP-Beni|NNP', 'Hssen|NNP'): 1,
 ('waystations|NNS', 'shuku|NN-eki|NN'): 1,
 ('Edinson|NNP', 'Cavani|NNP'): 1,
 ('Mirella|NNP', 'Freni|NNP'): 1,
 ('Flèche|NNP', 'Wallonne|NNP'): 1,
 ('Helicobacter|NNP', 'pylori|NNP'): 1,
 ('Sheela|NNP', 'Shrinivas|NNP'): 1,
 ('Mesut|NNP', 'Özil|NNP'): 1,
 ('Bundesarchiv|NNP', 'Bild|NNP'): 1,
 ('Sonu|NNP', 'Nigam|NNP'): 1,
 ('Päijänne|NNP', 'Tavastia|NNP'): 1,
 ('Bellas|NNP', 'Artes|NNP'): 1,
 ('Lemony|NNP', 'Snicket|NNP'): 1,
 ('Trisha|NNP', 'Yearwood|NNP'): 1,
 ('Pim|NNP', 'Fortuyn|NNP'): 1,
 ('Dany|NNP', 'Heatley|NNP'): 1,
 ('BNP|NNP', 'Paribas|NNP'): 1,
 ('Jomo|NNP', 'Kenyatta|NNP'): 1,
 ('Liev|NNP', 'Schreiber|NNP'): 1,
 ('Bertolt|NNP', 'Brecht|NNP'): 1,
 ('conventionally|RB', 'delimit|VBP'): 1,
 ('Rede|NNP', 'Globo|NNP'): 1,
 ('Dukla|NNP', 'Jihlava|NNP'): 1,
 ('Primorje|NNP-Gorski|NNP', 'Kotar|NNP'): 1,
 ('Lieke|NNP', 'Marten

In [61]:
len(batches)

150000

In [62]:
# Should be able to add batch information using df.map() but am encountering errors apparently relating
#  to indexing - workaround (though slower).

ngb_cols = ["ngram", "batch"]
rows = []

for ng in ngram_eval['ngram']:
    rows.append({"ngram" : ng,
                "batch" : batches[ng]})
    
ng_batch = pd.DataFrame(rows, columns = ngb_cols)

ng_batch

,ngram,batch
0,"(Grădina|NNP, Zoologică|NNP)",1
1,"(Ving|NNP, Rhames|NNP)",1
2,"(Karlovy|NNP, Vary|NNP)",1
3,"(Gharb|NNP-Chrarda|NNP-Beni|NNP, Hssen|NNP)",1
4,"(waystations|NNS, shuku|NN-eki|NN)",1
...,...,...
149995,"(supporters|NNS, of|IN, the|DT)",4
149996,"(episodes|NNS, and|CC)",3
149997,"(as|IN, old|JJ, as|IN)",-1
149998,"(John|NNP, Kerry|NNP)",1


In [63]:
ngram_eval = ngram_eval.merge(ng_batch, on='ngram')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Grădina|NNP, Zoologică|NNP)",20,-605.961128,2,1
1,"(Ving|NNP, Rhames|NNP)",20,-605.961128,2,1
2,"(Karlovy|NNP, Vary|NNP)",20,-605.961128,2,1
3,"(Gharb|NNP-Chrarda|NNP-Beni|NNP, Hssen|NNP)",20,-605.961128,2,1
4,"(waystations|NNS, shuku|NN-eki|NN)",20,-607.368914,2,1
...,...,...,...,...,...
149995,"(supporters|NNS, of|IN, the|DT)",47,-3262.258611,3,4
149996,"(episodes|NNS, and|CC)",64,-3262.274573,2,3
149997,"(as|IN, old|JJ, as|IN)",48,-3262.394255,3,-1
149998,"(John|NNP, Kerry|NNP)",77,-3262.495991,2,1


In [64]:
ngram_eval.to_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_eval_clean.pkl')

In [65]:
ngram_eval.batch.value_counts()

 1     47463
 2     33035
 3     24270
 4     16305
 5     10058
 6      7289
 7      4847
 8      2491
 9      1465
 10      942
-2       814
 11      457
-1       186
 12      171
 13      118
 14       65
 15       24
Name: batch, dtype: int64

## Extra filter to remove proper nouns

In [36]:
_pnouns = re.compile(".+\|NNPS?('|'s)?$")  # All tagged NNP/NNPS (proper noun/plural), NNP's / NNPS'
# NB: some things could get through - ('Dizzee|NNP', 'Rascal|NN'), ('Get|VB', 'Smart|JJ'), ("Assassin|NN's", 'Creed|NN')
# Also hits titles (Grand Vizier)

In [35]:
testt = ("Trek|NNPS'", 'Enterprise|NNS')

tupallmatch(testt, _pnouns)

False

In [37]:
# 208,344
ngrams_filt2 = ngrams_filt[~ngrams_filt.ngram.apply(tupallmatch , args= [_pnouns])]
# 190,157

In [38]:
ngrams_filt2 = ngrams_filt2.reset_index(drop=True)

ngrams_filt2

,ngram,freq,poisson,len
0,"(waystations|NNS, shuku|NN-eki|NN)",20,-6.073689e+02,2
1,"(conventionally|RB, delimit|VBP)",20,-6.381373e+02,2
2,"(mutually|RB, intelligible|JJ)",20,-6.415883e+02,2
3,"(ottava|NN, rima|NN)",21,-6.499528e+02,2
4,"(spiny|JJ, dogfish|NN)",20,-6.552806e+02,2
...,...,...,...,...
190152,"(and|CC, the|DT)",50301,-2.522274e+06,2
190153,"(to|IN, the|DT)",62524,-3.011091e+06,2
190154,"(is|VBZ, a|DT)",83635,-3.947372e+06,2
190155,"(in|IN, the|DT)",157323,-7.612996e+06,2


In [39]:
eval_count = 150000

ngram_eval2 = ngrams_filt2[0:eval_count]

ngram_eval2

,ngram,freq,poisson,len
0,"(waystations|NNS, shuku|NN-eki|NN)",20,-607.368914,2
1,"(conventionally|RB, delimit|VBP)",20,-638.137312,2
2,"(mutually|RB, intelligible|JJ)",20,-641.588322,2
3,"(ottava|NN, rima|NN)",21,-649.952795,2
4,"(spiny|JJ, dogfish|NN)",20,-655.280617,2
...,...,...,...,...
149995,"(He|PRP, wrote|VBD, about|IN)",67,-4314.528245,3
149996,"(for|IN, every|DT, 1000|CD)",72,-4314.701644,3
149997,"(a|DT, combined|VBN)",89,-4314.758077,2
149998,"(on|IN, July|NNP, 28|CD)",66,-4315.075053,3


In [43]:
batches, batch_count = batcher(ngram_eval2.ngram, stopwords=stop, max_batches = 15)

batches

{('waystations|NNS', 'shuku|NN-eki|NN'): 1,
 ('conventionally|RB', 'delimit|VBP'): 1,
 ('mutually|RB', 'intelligible|JJ'): 1,
 ('ottava|NN', 'rima|NN'): 1,
 ('spiny|JJ', 'dogfish|NN'): 1,
 ('white|JJ-headed|VBN', 'capuchin|NN'): 1,
 ('Blazing|VBG', 'Saddles|NNS'): 1,
 ('→|SYM', 'ℝ|NN'): 1,
 ('Elon|NNP', 'Musk|NN'): 1,
 ('Blink|NNP', '-182|CD'): 1,
 ('hawk|NN', 'Buteo|NNP'): 1,
 ('habeas|FW', 'corpus|FW'): 1,
 ('Dizzee|NNP', 'Rascal|NN'): 1,
 ('ukiyo|NN-e|NN', 'woodblock|NN'): 1,
 ('Charts|VBZ', 'Weekly|JJ'): 1,
 ('slime|NN', 'moulds|NNS'): 1,
 ('JPN|NN', 'KOR|NN'): 1,
 ('corpora|NN', 'cavernosa|NN'): 1,
 ('EVs|NNS', 'Romney|NNP'): 1,
 ("Assassin|NN's", 'Creed|NN'): 1,
 ('voir|FW', 'dire|FW'): 1,
 ('subfamily|NN', 'Asclepiadoideae|NNP'): 1,
 ('sprint|NN', 'canoeist|NN'): 1,
 ('Towering|VBG', 'Inferno|NN'): 1,
 ('-03|NN', 'Captain|NN'): 1,
 ('cognitive|JJ', 'neuropsychology|NN'): 1,
 ('Cosworth|NNP', 'CA2010|NN'): 1,
 ('Playoff|NNP', 'bracket|NN'): 1,
 ('Cartesian|JJ', 'coordinate|NN'): 

In [44]:
# Should be able to add batch information using df.map() but am encountering errors apparently relating
#  to indexing - workaround (though slower).

ngb_cols = ["ngram", "batch"]
rows = []

for ng in ngram_eval2['ngram']:
    rows.append({"ngram" : ng,
                "batch" : batches[ng]})
    
ng_batch = pd.DataFrame(rows, columns = ngb_cols)

ng_batch

,ngram,batch
0,"(waystations|NNS, shuku|NN-eki|NN)",1
1,"(conventionally|RB, delimit|VBP)",1
2,"(mutually|RB, intelligible|JJ)",1
3,"(ottava|NN, rima|NN)",1
4,"(spiny|JJ, dogfish|NN)",1
...,...,...
149995,"(He|PRP, wrote|VBD, about|IN)",3
149996,"(for|IN, every|DT, 1000|CD)",8
149997,"(a|DT, combined|VBN)",1
149998,"(on|IN, July|NNP, 28|CD)",1


In [45]:
ngram_eval2 = ngram_eval2.merge(ng_batch, on='ngram')

ngram_eval2

,ngram,freq,poisson,len,batch
0,"(waystations|NNS, shuku|NN-eki|NN)",20,-607.368914,2,1
1,"(conventionally|RB, delimit|VBP)",20,-638.137312,2,1
2,"(mutually|RB, intelligible|JJ)",20,-641.588322,2,1
3,"(ottava|NN, rima|NN)",21,-649.952795,2,1
4,"(spiny|JJ, dogfish|NN)",20,-655.280617,2,1
...,...,...,...,...,...
149995,"(He|PRP, wrote|VBD, about|IN)",67,-4314.528245,3,3
149996,"(for|IN, every|DT, 1000|CD)",72,-4314.701644,3,8
149997,"(a|DT, combined|VBN)",89,-4314.758077,2,1
149998,"(on|IN, July|NNP, 28|CD)",66,-4315.075053,3,1


In [47]:
ngram_eval2.to_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_eval_nopn.pkl')

In [48]:
ngram_eval2.batch.value_counts()

 1     45750
 2     35000
 3     21918
 4     16889
 5      9401
 6      7370
 7      4178
 8      4003
 9      2009
-2      1001
 10      827
 11      575
 12      424
-1       369
 13      179
 14       77
 15       30
Name: batch, dtype: int64